In [ ]:
# 使用新的配置模块进行优化实验
from optuna_config import create_robust_study, safe_optimize
from tuning_pipeline import optuna_log_manager, tune_pipeline

# 方法1: 创建新的研究
study_name = "backtest_tuning_with_side_auc_and_meta_f1"  # 固定名称，便于后续继续

with optuna_log_manager.verbose_optimization():
    study = create_robust_study(
        study_name=study_name,
        storage_dir="optuna_storage",
        direction="maximize",
        n_startup_trials=5000,
        percentile_for_pruning=10.0,
    )

    # 使用安全优化函数
    safe_optimize(
        study=study,
        objective=tune_pipeline,
        n_trials=25000,
        n_jobs=1,
        gc_after_trial=True,
        show_progress_bar=False,
    )

In [ ]:
sorted(study.trials, key=lambda t: t.value if t.value is not None else 0, reverse=True)[
    :3
]

In [ ]:
# 方法2: 继续之前中断的实验
from optuna_config import OptunaStorageManager, safe_optimize
from tuning_pipeline import optuna_log_manager, tune_pipeline

# 使用存储管理器
manager = OptunaStorageManager("optuna_storage")

# 列出所有可用的研究
studies = manager.list_studies()
if studies:
    print("可用的研究:")
    for i, summary in enumerate(studies):
        print(f"  {i + 1}. {summary.study_name}: {summary.n_trials} 个试验")

        # 方法2a: 继续特定的研究
        study_name_to_continue = "backtest_tuning_main"  # 指定要继续的研究名称

        if manager.study_exists(study_name_to_continue):
            print(f"\n继续研究: {study_name_to_continue}")

            with optuna_log_manager.verbose_optimization():
                # 直接加载指定的研究
                study = manager.create_or_load_study(
                    study_name=study_name_to_continue, load_if_exists=True
                )

                # 显示当前状态
                info = manager.get_study_info(study_name_to_continue)
                print(f"研究信息: {info}")

                # 继续优化
                safe_optimize(
                    study=study,
                    objective=tune_pipeline,
                    n_trials=50000,
                    n_jobs=1,
                    gc_after_trial=True,
                )
        else:
            print(f"研究 '{study_name_to_continue}' 不存在")

        # 方法2b: 继续最新的研究
        # latest_study = manager.get_latest_study()
        # if latest_study:
        #     print(f"继续最新的研究: {latest_study.study_name}")

        #     with optuna_log_manager.verbose_optimization():
        #         safe_optimize(
        #             study=latest_study,
        #             objective=tune_pipeline,
        #             n_trials=50000,
        #             n_jobs=1,
        #             gc_after_trial=True,
        #         )
        # else:
        #     print("没有找到任何研究，请先运行第一个cell创建新研究")
else:
    print("没有找到任何研究，请先运行第一个cell创建新研究")